In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import WandbLogger
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
sys.path.append('/data/om/reflection_anydoor/datasets')
sys.path.append('/data/om/reflection_anydoor')


from mirrors import MirrorsDataset
from cldm.logger import ImageLogger
from cldm.model import create_model, load_state_dict
from torch.utils.data import ConcatDataset
from cldm.hack import disable_verbosity, enable_sliced_attention
from omegaconf import OmegaConf

save_memory = False
disable_verbosity()
if save_memory:
    enable_sliced_attention()

# Configs
resume_path = '/data/om/reflection_anydoor/checkpoints/counterfactual_training_epoch=106-step=25572.ckpt'
batch_size = 4
logger_freq = 1000
learning_rate = 1e-5
sd_locked = False
only_mid_control = False
n_gpus = 1
accumulate_grad_batches=1

# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('./configs/anydoor.yaml').cpu()
model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control

# Datasets
DConf = OmegaConf.load('./configs/datasets.yaml')


In [ ]:
dataset = MirrorsDataset(**DConf.Train.Mirrors)
dataloader = DataLoader(dataset, num_workers=4, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)

In [ ]:
import wandb
from datetime import datetime
wandb.login(key='39e65ab86c39c92f1b18458c6cc56fee691e0705')
wandb.init(project="anydoor_inpainting")
wandb.run.name = datetime.now().strftime("%Y-%m-%d_%H%M%S")
wandb_logger = WandbLogger()

In [ ]:
trainer = pl.Trainer(
    gpus=[0], 
    precision=16, 
    accelerator="gpu", 
    # callbacks=[logger], 
    progress_bar_refresh_rate=1, 
    accumulate_grad_batches=accumulate_grad_batches,
    logger=wandb_logger,
    )

# Train!
trainer.fit(model, dataloader)


In [1]:
import torch
random_tensor = torch.rand(3, 64, 64)
random_tensor.unsqueeze(1).shape

torch.Size([3, 1, 64, 64])